In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor

pd.options.display.float_format = '{:.2f}'.format

import warnings                 # to deal with warning messages
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("/kaggle/input/chennai-housing-sales-price/Chennai houseing sale.csv")
df.head()

In [ ]:
df1=df.drop(['QS_OVERALL','QS_BEDROOM','QS_BATHROOM','QS_ROOMS','MZZONE'],axis='columns')
df1.head()

In [ ]:
df1.info()

In [ ]:
df1.isnull().sum()

In [ ]:
df2=df1.dropna()
df2.isnull().sum()

In [ ]:
for col in df.columns:
    if df[col].dtype=='object':
        print()
        print(col)
        print(df[col].unique())

In [ ]:
df2.AREA.replace(['Ana Nagar','Ann Nagar'],'Anna Nagar',inplace=True)
df2.AREA.replace('Karapakkam','Karapakam',inplace=True)
df2.AREA.replace('Adyr','Adyar',inplace=True)
df2.AREA.replace('Velchery','Velachery',inplace=True)
df2.AREA.replace(['Chormpet','Chrmpet','Chrompt'],'Chrompet',inplace=True)
df2.AREA.replace('KKNagar','KK Nagar',inplace=True)
df2.AREA.replace('TNagar','T Nagar',inplace=True)
df2.SALE_COND.replace('Ab Normal','AbNormal',inplace=True)
df2.SALE_COND.replace(['PartiaLl','Partiall'],'Partial',inplace=True)
df2.SALE_COND.replace('AdjLand','Adj Land',inplace=True)
df2.PARK_FACIL.replace('Noo','No',inplace=True)
df2.BUILDTYPE.replace('Comercial','Commercial',inplace=True)
df2.BUILDTYPE.replace('Other','Others',inplace=True)
df2.UTILITY_AVAIL.replace('NoSewr','NoSeWa',inplace=True)
df2.UTILITY_AVAIL.replace('AllPub','All Pub',inplace=True)
df2.STREET.replace('Pavd','Paved',inplace=True)
df2.STREET.replace('NoAccess','No Access',inplace=True)

In [ ]:
df2.info()

In [ ]:
df2['total_price']=df2['REG_FEE']+df2['COMMIS']+df2['SALES_PRICE']

In [ ]:
df2['DATE_SALE'] = pd.to_datetime(df2['DATE_SALE'],format='%d-%m-%Y')
df2['DATE_BUILD'] = pd.to_datetime(df2['DATE_BUILD'],format='%d-%m-%Y')

In [ ]:
df2['AGE'] = pd.DatetimeIndex(df2['DATE_SALE']).year - pd.DatetimeIndex(df2['DATE_BUILD']).year

In [ ]:
df2['N_BEDROOM']=df2['N_BEDROOM'].astype(int)
df2['N_BATHROOM']=df2['N_BATHROOM'].astype(int)

In [ ]:
import seaborn as sns

In [ ]:
df2.describe()

In [ ]:
df3=df2.drop(['SALES_PRICE','COMMIS','REG_FEE','DATE_BUILD','DATE_SALE',],axis='columns')
df3.head()

In [ ]:
columns_to_analyze = ['INT_SQFT','DIST_MAINROAD','N_BEDROOM','N_BATHROOM','N_ROOM','total_price','AGE']

plt.figure(figsize=(16, 8))

for i, column in enumerate(columns_to_analyze):
    plt.subplot(2, 4, i + 1)
    sns.boxplot(x=df3[column])
    plt.title(column)
    plt.tight_layout()

plt.show()

In [ ]:
Q1 = df3["total_price"].quantile(0.25)
Q3 = df3["total_price"].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
lower_threshold = Q1 - 1.5 * IQR
upper_threshold = Q3 + 1.5 * IQR

In [ ]:
outlier_mask = (df3["total_price"] < lower_threshold) | (df3["total_price"] > upper_threshold)

In [ ]:
df3_cleaned = df3[~outlier_mask]

In [ ]:
df4=df3_cleaned.copy()

In [ ]:
columns_to_analyze = ['INT_SQFT','DIST_MAINROAD','N_BEDROOM','N_BATHROOM','N_ROOM','total_price','AGE']

plt.figure(figsize=(16, 8))

for i, column in enumerate(columns_to_analyze):
    plt.subplot(2, 4, i + 1)
    sns.boxplot(x=df4[column])
    plt.title(column)
    plt.tight_layout()

plt.show()

In [ ]:
sns.pairplot(df4)
plt.show()

In [ ]:
df4.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
columns_to_encode = ['AREA','SALE_COND','PARK_FACIL','BUILDTYPE','UTILITY_AVAIL','STREET']
encoded_df = pd.get_dummies(df4[columns_to_encode],dtype=int)

In [ ]:
encoded_df.head()

In [ ]:
df5=pd.concat([df4,encoded_df],axis=1)
df5.head()

In [ ]:
df6=df5.copy()

In [ ]:
#Train test split
y=df6['total_price']
X=df6.drop(['PRT_ID','SALE_COND','PARK_FACIL','BUILDTYPE','UTILITY_AVAIL','STREET','AREA','total_price'],axis='columns')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

In [ ]:
y_pred=lr_clf.predict(X_test)

In [ ]:
out=pd.DataFrame({'Price_actual':y_test,'Price_pred':y_pred})
result=df6.merge(out,left_index=True,right_index=True)

In [ ]:
result[['PRT_ID','AREA','Price_actual','Price_pred']].sample(20)

# Thank you.